
# Autoencoders

Autoencoders are a type of artificial neural network used for unsupervised learning tasks, particularly for dimensionality reduction, feature learning, and data compression. They consist of two main components:

-   **Encoder**: Compresses the input data into a lower-dimensional representation (latent space).
-   **Decoder**: Reconstructs the original data from the compressed representation.

Autoencoders are trained to minimize the difference between the input and the reconstructed output, effectively learning to capture the essential features of the data while discarding noise and irrelevant information.

## Example: Autoencoder on MNIST Dataset

To illustrate the use of autoencoders, we can apply them to the MNIST dataset, which consists of handwritten digits. The goal is to compress the images into a lower-dimensional representation and then reconstruct the original images.

-   Load the MNIST dataset

In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = X_train.reshape((X_train.shape[0], 28, 28, 1)).astype('float32') / 255.0
X_test = X_test.reshape((X_test.shape[0], 28, 28, 1)).astype('float32') / 255.0
y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)
X_train.shape, X_test.shape

-   Visualize some samples from the dataset

In [ ]:
# Visualize some samples from the dataset
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 4))
for i in range(10):
    plt.subplot(2, 5, i + 1)
    plt.imshow(X_train[i].reshape(28, 28), cmap='gray')
    plt.title(f'Label: {y_train[i]}')
    plt.axis('off')
plt.tight_layout()
plt.show()

-   Define the autoencoder architecture

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Flatten, Reshape

# Define the encoder
input_img = Input(shape=(28, 28, 1))
x = Flatten()(input_img)
encoded = Dense(64, activation='relu')(x)

# Define the decoder
x = Dense(28 * 28 * 1, activation='sigmoid')(encoded)
decoded = Reshape((28, 28, 1))(x)

# Create the autoencoder model
autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')
autoencoder.summary()

-   Train the autoencoder

In [ ]:
# Train the autoencoder
autoencoder.fit(X_train,
                X_train,
                epochs=50,
                batch_size=256,
                shuffle=True,
                validation_split=0.2)
print("Autoencoder training completed.")

-   Evaluate the autoencoder

In [ ]:
# Evaluate the autoencoder
import numpy as np
reconstruction = autoencoder.predict(X_test)
reconstruction_error = np.mean(np.square(X_test - reconstruction))
print(f'Reconstruction Error: {reconstruction_error:.4f}')

-   Visualize the reconstructed images

In [ ]:
# Visualize the original and reconstructed images
n = 10  # Number of images to visualize
plt.figure(figsize=(10, 4))
for i in range(n):
    # Original image
    plt.subplot(2, n, i + 1)
    plt.imshow(X_test[i].reshape(28, 28), cmap='gray')
    plt.title(f'Original: {y_test[i]}')
    plt.axis('off')

    # Reconstructed image
    plt.subplot(2, n, i + n + 1)
    plt.imshow(reconstruction[i].reshape(28, 28), cmap='gray')
    plt.title('Reconstructed')
    plt.axis('off')
plt.tight_layout()
plt.show()

-   Save the autoencoder model

In [ ]:
# Save the autoencoder model
autoencoder.save('autoencoder_mnist.h5')
print("Autoencoder model saved as 'autoencoder_mnist.h5'.")

## Hands-on Exercise

Repeat the above steps with the CIFAR-10 dataset, which consists of 32x32 color images in 10 classes. The goal is to compress the images into a lower-dimensional representation and then reconstruct the original images.

-   Load the CIFAR-10 dataset

In [ ]:
from tensorflow.keras.datasets import cifar10
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0
y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)
X_train.shape, X_test.shape

-   Visualize some samples from the CIFAR-10 dataset

In [ ]:
# Visualize some samples from the CIFAR-10 dataset
plt.figure(figsize=(10, 4))
for i in range(10):
    plt.subplot(2, 5, i + 1)
    plt.imshow(X_train[i])
    plt.title(f'Label: {y_train[i]}')
    plt.axis('off')
plt.tight_layout()
plt.show()

-   Define the autoencoder architecture for CIFAR-10

In [ ]:
# Define the encoder for CIFAR-10
input_img = Input(shape=(32, 32, 3))
# Add a Convolutional layer for better feature extraction
from tensorflow.keras.layers import Conv2D, MaxPooling2D
x = Conv2D(32, (3, 3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Flatten()(x)
encoded = Dense(256, activation='relu')(x)

# Define the decoder for CIFAR-10
x = Dense(32 * 32 * 3, activation='sigmoid')(encoded)
# Add a Convolutional transpose layer to reconstruct the image
from tensorflow.keras.layers import Reshape
x = Reshape((4, 4, 128))(x)  # Reshape to match the last pooling layer's output
x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)
# Reshape the output to match the original image dimensions
decoded = Reshape((32, 32, 3))(x)

# Create the autoencoder model for CIFAR-10
autoencoder_cifar = Model(input_img, decoded)
autoencoder_cifar.compile(optimizer='adam', loss='binary_crossentropy')
autoencoder_cifar.summary()

-   Train the autoencoder on CIFAR-10

In [ ]:
# Train the autoencoder on CIFAR-10
autoencoder_cifar.fit(X_train,
                      X_train,
                      epochs=50,
                      batch_size=256,
                      shuffle=True,
                      validation_split=0.2)
print("Autoencoder for CIFAR-10 training completed.")

-   Evaluate the autoencoder on CIFAR-10

In [ ]:
# Evaluate the autoencoder on CIFAR-10
reconstruction_cifar = autoencoder_cifar.predict(X_test)
reconstruction_error_cifar = np.mean(np.square(X_test - reconstruction_cifar))
print(f'Reconstruction Error for CIFAR-10: {reconstruction_error_cifar:.4f}')

-   Visualize the reconstructed images from CIFAR-10

In [ ]:
# Visualize the original and reconstructed images from CIFAR-10
n = 10  # Number of images to visualize
plt.figure(figsize=(10, 4))
for i in range(n):
    # Original image
    plt.subplot(2, n, i + 1)
    plt.imshow(X_test[i])
    plt.title(f'Original: {y_test[i]}')
    plt.axis('off')

    # Reconstructed image
    plt.subplot(2, n, i + n + 1)
    plt.imshow(reconstruction_cifar[i])
    plt.title('Reconstructed')
    plt.axis('off')
plt.tight_layout()
plt.show()